In [3]:
import pandas as pd
import numpy as np
import time

# List of gene associeated with selected disease

In [4]:
data_first = pd.read_csv('curated_gene_disease_associations.tsv', sep='\t')
data_first.head()

,geneId,geneSymbol,DSI,DPI,diseaseId,diseaseName,diseaseType,diseaseClass,diseaseSemanticType,score,EI,YearInitial,YearFinal,NofPmids,NofSnps,source
0,1,A1BG,0.700,0.538,C0019209,Hepatomegaly,phenotype,C23;C06,Finding,0.30,1.000,2017.0,2017.0,1,0,CTD_human
1,1,A1BG,0.700,0.538,C0036341,Schizophrenia,disease,F03,Mental or Behavioral Dysfunction,0.30,1.000,2015.0,2015.0,1,0,CTD_human
2,2,A2M,0.529,0.769,C0002395,Alzheimer's Disease,disease,C10;F03,Disease or Syndrome,0.50,0.769,1998.0,2018.0,3,0,CTD_human
3,2,A2M,0.529,0.769,C0007102,Malignant tumor of colon,disease,C06;C04,Neoplastic Process,0.31,1.000,2004.0,2019.0,1,0,CTD_human
4,2,A2M,0.529,0.769,C0009375,Colonic Neoplasms,group,C06;C04,Neoplastic Process,0.30,1.000,2004.0,2004.0,1,0,CTD_human


here we're taking only genes associated with "epileptic drop attack"

In [5]:
data_sel = data_first[data_first['diseaseId'] =='C0270846']

In [6]:
sel_to_join = data_sel.drop(['DSI', 'DPI', 'diseaseId', 'diseaseName',
       'diseaseType', 'diseaseClass', 'diseaseSemanticType', 'score', 'EI',
       'YearInitial', 'YearFinal', 'NofPmids', 'NofSnps', 'source'], axis=1)

this is a simple check to understand which is the right "geneId" of gene named "CHRNA7" because in uniprotKB there are two different "geneID" for this gene

In [5]:
sel_to_join[sel_to_join['geneId']==1139]

,geneId,geneSymbol
9900,1139,CHRNA7


here we're creating the list of "geneSymbol" in order to use it in HGNC online tool

In [6]:
symbol_list = list(sel_to_join['geneSymbol'])

In [7]:
id_list = list(sel_to_join['geneId'])

# HGNC check

In this part, we are sorting the symbols to get only the approved names, and withdraw the "alias", or "previous names".
We are also putting all of them in a file to be able to pass it to the UNIPROT website.

In [8]:
file_name = 'hgnc-symbol-check_approved.csv'
data_name = pd.read_csv(file_name)

saving the file in order to use it with uniprot online tool

In [9]:
data_name['Approved symbol'].to_csv('uniprot_entry.txt', index=False, header=False)

# UNIPROT collection of data

In this part we are getting specific informations from the UNIPROT website, linked to each genes with an official symbol. We are using the GeneID, the Protein name, the function,...
\
Actually, we saw that among all the genes, one don't have any function referenced in UNIPROT: gene PLPPR1.
\
According to the website GeneCard: https://www.genecards.org/cgi-bin/carddisp.pl?gene=PLPPR1 , this gene has several other alias. One of them is PRG3, which is also an approved name. When checked, this one has a function in UNIPROT. 
\
But when we checked its GeneID with the one given by the informations in the previous website, the GeneID are different. Thus in the rest of the project, we chose to keep the name 'PLPPR1'

In [7]:
# cleaning data coming from uniprotKB
data = pd.read_excel('uniprot_list.xlsx')
data = data.rename(columns={'yourlist:M20201212216DA2B77BFBD2E6699CA9B6D1C41EB207E9E3I': 'Approved name'})
data['Cross-reference (GeneID)'] = data['Cross-reference (GeneID)'].apply(lambda x: int(x.replace(';', '')))
data.loc[data['Cross-reference (GeneID)'] == 113989832, 'Cross-reference (GeneID)'] = 1139
data = data[data['Cross-reference (GeneID)'].isin(sel_to_join['geneId'])]
data['Protein names'] = data['Protein names'].apply(lambda x: str(x).split(',')[0])
data['Function [CC]'] = data['Function [CC]'].apply(lambda x: str(x)[0:70] + '...')
data = data.drop(['Entry name', 'Gene names  (primary )', 'Status' ], axis=1)
data['Protein names'] = data['Protein names'].apply(lambda x: str(x).split(' (')[0])
data['Protein names'] = data['Protein names'].apply(lambda x: str(x).split('[')[0])

In [11]:
data.to_csv('uniprot_data_collection.csv', index=False)

# BioGrid Human (interactions)

In [8]:
# list of 101 seed genes used to filter biogrid interactions
list_names = list(data['Approved name'])

Here we're saving a file("network_file.txt") with all the human/human interactions in Biogrid Human in order to use it in DIAMOND tool

In [9]:
biogrid = pd.read_csv('BIOGRID-ORGANISM-Homo_sapiens-4.2.191.tab3.txt', delimiter='\t')

F:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
net_file = biogrid[(biogrid['Organism ID Interactor A'] == 9606) & (biogrid['Organism ID Interactor B'] == 9606)]

In [15]:
net_file = net_file.drop(['Entrez Gene Interactor A',
       'Entrez Gene Interactor B', '#BioGRID Interaction ID', 'BioGRID ID Interactor A',
       'BioGRID ID Interactor B', 'Systematic Name Interactor A',
       'Systematic Name Interactor B', 'Synonyms Interactor A',
       'Synonyms Interactor B', 'Experimental System',
       'Experimental System Type', 'Author', 'Publication Source',
       'Organism ID Interactor A', 'Organism ID Interactor B', 'Throughput',
       'Score', 'Modification', 'Qualifications', 'Tags', 'Source Database',
       'SWISS-PROT Accessions Interactor A', 'TREMBL Accessions Interactor A',
       'REFSEQ Accessions Interactor A', 'SWISS-PROT Accessions Interactor B',
       'TREMBL Accessions Interactor B', 'REFSEQ Accessions Interactor B',
       'Ontology Term IDs', 'Ontology Term Names', 'Ontology Term Categories',
       'Ontology Term Qualifier IDs', 'Ontology Term Qualifier Names',
       'Ontology Term Types', 'Organism Name Interactor A',
       'Organism Name Interactor B'], axis=1)

In [16]:
net_file.to_csv('network_file.txt', index=False, header=False)

Here we creating our disease interactome file("net_edge.csv") in order to use it to create our network with networkx

In [10]:
# JUST HOMO SAPIENS/HOMO SAPIENS INTERACTIONS
biogrid = biogrid[(biogrid['Organism ID Interactor A'] == 9606) & (biogrid['Organism ID Interactor B'] == 9606)]

In [11]:
# JUST SEED GENE INTERACTIONs
biogrid_final = biogrid[(biogrid['Official Symbol Interactor A'].isin(list_names)) | (biogrid['Official Symbol Interactor B'].isin(list_names))]

Here we have the number of genes in the disease interactome

In [12]:
complete_list = list(biogrid_final['Official Symbol Interactor A']) + list(biogrid_final['Official Symbol Interactor B'])
complete_list = list(np.unique(complete_list))
len(complete_list)

3923

here we're creating the file of all disease interactome genes in order to perform the enrichment analysis

In [13]:
with open('interactome_genes.txt', 'w') as filehandle:
    for listitem in complete_list:
        filehandle.write('%s\n' % listitem)

Here we're checking the name of the only seed genes which does not appear in Biogrid human

In [20]:
set(list_names) - set(complete_list)

{'PLPPR1'}

Here we have the number of interactions in the disease interactome

In [21]:
biogrid_final = biogrid[(biogrid['Official Symbol Interactor A'].isin(complete_list)) & (biogrid['Official Symbol Interactor B'].isin(complete_list))]
print(len(biogrid_final))

183747


Now we're creating the disease interactome file

In [22]:
biogrid_final = biogrid_final.drop(['Entrez Gene Interactor A',
       'Entrez Gene Interactor B', '#BioGRID Interaction ID', 'BioGRID ID Interactor A',
       'BioGRID ID Interactor B', 'Systematic Name Interactor A',
       'Systematic Name Interactor B', 'Synonyms Interactor A',
       'Synonyms Interactor B', 'Experimental System',
       'Experimental System Type', 'Author', 'Publication Source',
       'Organism ID Interactor A', 'Organism ID Interactor B', 'Throughput',
       'Score', 'Modification', 'Qualifications', 'Tags', 'Source Database',
       'SWISS-PROT Accessions Interactor A', 'TREMBL Accessions Interactor A',
       'REFSEQ Accessions Interactor A', 'SWISS-PROT Accessions Interactor B',
       'TREMBL Accessions Interactor B', 'REFSEQ Accessions Interactor B',
       'Ontology Term IDs', 'Ontology Term Names', 'Ontology Term Categories',
       'Ontology Term Qualifier IDs', 'Ontology Term Qualifier Names',
       'Ontology Term Types', 'Organism Name Interactor A',
       'Organism Name Interactor B'], axis=1)

In [23]:
biogrid_final.to_csv('net_edge.csv', index=False)

# PART 2

The interactome of interest (built by filtering the complete interactome coming from BioGrid) is stored in the file "net_edge.csv" as an edge list, and is used to build a Graph object with the library NetworkX, which is used to compute all measures and operations on the interactome.

In [29]:
biogrid_final = pd.read_csv('net_edge.csv')
edges = []
for i in range(len(biogrid_final)):
    edges.append((biogrid_final['Official Symbol Interactor A'][i], biogrid_final['Official Symbol Interactor B'][i]))

In [30]:
import networkx as nx
G = nx.Graph()    

In [31]:
G.add_edges_from(edges)

# Global measures

In [31]:
nn = nx.number_of_nodes(G)
ne = nx.number_of_edges(G)
AvgSP = nx.average_shortest_path_length(G)
AvgCl = nx.average_clustering(G)
nDegree = G.degree()

density = 2*ne/(nn*(nn+1))
values = dict(nDegree).values()
max_k = max(values)
centralization = (nn/(nn-2))*(max_k/(nn+1)) - density

print(f"Number of nodes: {nn}\n")

print(f"Number of links: {ne}\n")

print(f"Number of isolated nodes: {nx.number_of_isolates(G)}\n")

print(f"Average path length between nodes: {AvgSP}\n")

print(f"Average degree of the interactome: {ne/nn}\n")

print(f"Average clustering coefficient: {AvgCl}\n")

print(f"Diameter: {nx.diameter(G)}, Radius: {nx.radius(G)}\n")

print(f"Centralization of the graph: {centralization}")

Number of nodes: 3923

Number of links: 130563

Number of isolated nodes: 0

Average path length between nodes: 2.422625598872118

Average degree of the interactome: 33.28141728269182

Average clustering coefficient: 0.185725717191852

Diameter: 6, Radius: 3

Centralization of the graph: 0.30353678093864594


In [38]:
#nx.draw(G)

# LCC measures

In [32]:
LCC = max(nx.connected_components(G), key=len)
LCC = G.subgraph(LCC)

In [33]:
print(f"Number of nodes of the LCC: {nx.number_of_nodes(LCC)}\n")
print(f"Number of edges of the LCC: {nx.number_of_edges(LCC)}\n")

Number of nodes of the LCC: 3923

Number of edges of the LCC: 130563



Since the number of isolated nodes is 0, the LCC is exactly the complete graph, so all the measures remain the same.

### Local measures of centrality

The betweenness centrality of a node is the number of shortest paths of the graph passing through that node. 

In [34]:
# Here we create a pandas DataFrame with betweenness centrality 
# and degree for the first 20 nodes with the highest betweennes centrality

bet = nx.betweenness_centrality(LCC)
bet = pd.DataFrame.from_dict(bet, orient='index', columns=['betweness centrality'])
deg = pd.DataFrame.from_dict(dict(nDegree), orient='index', columns=['node degree'])
dat = bet.sort_values(by='betweness centrality', axis=0, ascending=False).head(20)
dat = dat.join(deg)

The eigenvector centrality measures the centrality of a node based on the centrality of its neighbours. This is done, as the name suggests, through eigenvectors of the adjacency matrix. 

In [35]:
eigen = nx.eigenvector_centrality(LCC, max_iter= 500)
eig = pd.DataFrame.from_dict(eigen, orient='index', columns=['eigenvector centrality'])

In [36]:
two = dat.join(eig)

Closeness central of a node is the reciprocal of the sum of the length of the shortest paths between the node and all other nodes in the graph. Thus, the more central a node is, the closer it is to all other nodes.

In [37]:
clos = nx.closeness_centrality(LCC)
clos = pd.DataFrame.from_dict(clos, orient='index', columns=['closeness centrality'])

In [38]:
three = two.join(clos)

In [48]:
bet_over_deg = dat['betweness centrality']/dat['node degree']
bet_over_deg = {'betweenness/degree':bet_over_deg}
bet_over_deg = pd.DataFrame(bet_over_deg)
lcc_local = three.join(bet_over_deg)

The dataset containing the information about Node degree, Betweenness
centrality, Eigenvector centrality, Closeness centrality and Betweenness/Node degree
for the first twenty nodes with highest betweenness centrality is stored
in a csv file:

In [50]:
lcc_local.to_csv('lcc_local.csv')

# CLUSTERING

In [32]:
import markov_clustering as mc
from scipy.stats import hypergeom

In [33]:
matrix = nx.to_scipy_sparse_matrix(G)
# run MCL with default parameters
result = mc.run_mcl(matrix)          
clusters = mc.get_clusters(result) 

In [34]:
nodes = np.array(list(G.nodes()))
seed_list = list_names

After performing the MCL clustering, we had to take just clusters with number of element >= 10, and with a statistical overepresentation of seed genes(hypergeometric test with a p_value < 0.05). We also printed the requested informations in order to report them.
Results show us 4 cluster with the characteristics described before.
 

In [35]:
put_dis_mod = []
j = 1
for i in clusters:
    if len(i)>=10:
        clus = nodes[list(i)]
        K = len(nodes)
        k = len(seed_list)
        N = len(clus)
        n = len(set(clus).intersection(set(seed_list)))
        pval = hypergeom.sf(n, K, k, N)
        if pval < 0.05:
            print(pd.DataFrame(
                        index=[
                            f"Module {j}"],
                        data={
                            "N_total": N,
                            "n_seed": n,
                            "ratio": n/N,
                            "pval": pval,
                           }))
            j += 1
            put_dis_mod.append(clus)
        

          N_total  n_seed     ratio      pval
Module 1       11       1  0.090909  0.031027
          N_total  n_seed  ratio      pval
Module 2       10       1    0.1  0.025813
          N_total  n_seed  ratio      pval
Module 3       10       1    0.1  0.025813
          N_total  n_seed     ratio      pval
Module 4       11       1  0.090909  0.031027


In [36]:
put_dis_mod

[array(['RBFOX2', 'DAB1', 'CELF3', 'PLK2', 'RBFOX1', 'PCBP4', 'SIRPB1',
        'CCDC103', 'SDR9C7', 'RBFOX3', 'ZIC1'], dtype='<U14'),
 array(['GLRX3', 'MEOX2', 'AMOTL2', 'MMP25', 'BCAT2', 'KRT81', 'TGM1',
        'TEX14', 'CYP4F12', 'CHRNB4'], dtype='<U14'),
 array(['RASGRP3', 'PRKCD', 'PRKCE', 'PRKCQ', 'GABRA1', 'CARD11', 'IL32',
        'PPRC1', 'H2BFS', 'LYSMD2'], dtype='<U14'),
 array(['AMD1', 'TPSAB1', 'HLA-DRB3', 'POMC', 'MC4R', 'MC5R', 'MC3R',
        'TTL', 'HDDC2', 'TSPAN11', 'BHLHA9'], dtype='<U14')]

# DIAMOND TOOL



For this part we used the diamond tool by creating an environment with python 2.7 and by lunching "DIAMOnD.py PPI.txt seed_genes.txt 100" on the command line. In our case, PPI.txt was a comma separated file with all the human/human interactions found in Biogrid Human and seed_genes.txt 100 was a text file with our 101 seed gene symbols(one for each row). 




In [25]:
dimond = pd.read_csv('first_200_added_nodes_weight_1.txt', delimiter='\t')
dimond.head()

,#rank,DIAMOnD_node
0,1,GPRASP1
1,2,HRH3
2,3,HTR1D
3,4,VEGFA
4,5,CLIC6


In [37]:
# for i in dimond['DIAMOnD_node'][:30]:
#      print(i)